In [161]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
import requests
import os

TELEGRAM_TOKEN = "8083975934:AAEFXz0ja40Z470mmZSqGej7g6p80Kt6wh0"  # o ponlo directamente como string
TELEGRAM_CHAT_ID = "1030345806"

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
    payload = {
        "chat_id": TELEGRAM_CHAT_ID,
        "text": message
    }
    try:
        response = requests.post(url, data=payload)
        if response.status_code != 200:
            print("❌ Error en la respuesta de Telegram:")
            print(response.status_code)
            print(response.text)
        else:
            print("✅ Mensaje enviado correctamente.")
    except Exception as e:
        print(f"❌ No se pudo enviar el mensaje de Telegram: {e}")


In [163]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.utilities import metrics
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score
from financial.model import Model

import tensorflow as tf
from keras import backend as K
import gc

def reset_gpu():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

In [164]:
load_dotenv(dotenv_path=find_dotenv())

cache_path = os.environ["CACHE"]
model_path = os.environ["MODEL"]
print(model_path)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache_path+"/"))
print(datastore) 

/home/manidmt/TFG/OTRI/models/keras
/home/manidmt/TFG/OTRI/data
CachedDataStore with 960 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [165]:
start_date = "1990-01-01"
end_date = "2025-06-30"
data = datastore.get_data("AAPL", start_date, end_date)
print(data)

Date
1990-01-02      0.261499
1990-01-03      0.263254
1990-01-04      0.264132
1990-01-05      0.265009
1990-01-08      0.266764
                 ...    
2025-06-24    200.300003
2025-06-25    201.559998
2025-06-26    201.000000
2025-06-27    201.080002
2025-06-30    205.169998
Name: AAPL, Length: 8939, dtype: float64


In [166]:
import os
import pickle

def store_results(ticker: str, model_name: str, predictions: pd.Series):

    base_name = f"model-momentum-{model_name}@pred"

    # Guardar .pkl para Wrapper
    with open(os.path.join(cache_path, base_name), "wb") as f:
        pickle.dump(predictions, f)
    print("Prediciones guardadas pickle")
    # Guardar .csv para inspección y web
    predictions.to_csv(os.path.join(model_path, model_name + "_preds.csv"))
    print("Prediciones guardadas csv")
    

In [167]:
factory = KerasAdvancedModelFactory()

In [172]:
from itertools import product

# Arquitecturas a probar
architectures = ["rnn","cnn"]
#architectures = ["rnn","lstm", "cnn", "cnn2d", "transformer"]

# Hiperparámetros a explorar
layer_option = [64, 32]
activation = ["relu"]
batch_size = 32
epochs = 10

def ticker_single_models(ticker):

    data = datastore.get_data(ticker, start_date, end_date)    
    target = data[90 + 20:]
    
    for arch in architectures:
        name = f"{arch}_{ticker}_{end_date[:4]}_single"
        path = model_path + "/" + name

        try:
            if Model.from_file(name, path=os.environ["MODEL"]):
                print(f"Modelo {name} ya existe. Saltando...")
                #send_telegram_message(f"Modelo {name} ya existe. Saltando...")
                continue
        except FileNotFoundError:
            print(f"📂 Modelo {name} no existe. Procediendo a entrenar...")
            #send_telegram_message(f"Modelo {name} no existe. Procediendo a entrenar...")

        config = {
            "mode": "global",  
            "datastore": datastore,
            "ticker": [ticker],
            "model_factory": factory,
            "name": name,
            "start_year": start_date,
            "end_year": end_date,
            "lookahead": 20,
            "horizon": 90,
            "model_params": {
                "architecture": arch,
                "topology": {
                    "layers": layer_option,
                    "activation": {
                        "hidden": activation,
                        "output": "linear"
                    }
                },
                "optimization": {
                    "optimizer": "adam",
                    "loss": "mean_squared_error",
                    "metrics": ["mae"],
                    "epochs": epochs,
                    "batch_size": batch_size,
                    "validation_split": 0.1
                }
            }
        }

        try:
            print("RUNNING MODEL:")
            experiment = ModelExperimentFactory.create_experiment(config)
            experiment.run()
            print("METRICS:")
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            store_results(ticker, name, preds)
            send_telegram_message("Resultados guardados")
            metrics(experiment, preds, target, model_path)
            send_telegram_message("Metricas guardadas")
            reset_gpu()
        except Exception as e:
            print(f"Falló el modelo {name}: {e}")

In [173]:
ticker_single_models("AAPL")
ticker_single_models("BLK")
ticker_single_models("BAM")

📂 Modelo rnn_AAPL_2025_single no existe. Procediendo a entrenar...
RUNNING MODEL:
Multiple tickers detected: ['AAPL', 'BAM']. Using multi-ticker features.
Architecture: rnn
Epoch 1/300
Falló el modelo rnn_AAPL_2025_single: Exception encountered when calling SimpleRNNCell.call().

Dimensions must be equal, but are 1 and 90 for '{{node sequential_41_1/simple_rnn_45_1/simple_rnn_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_41_1/simple_rnn_45_1/strided_slice_2, sequential_41_1/simple_rnn_45_1/simple_rnn_cell_1/Cast/ReadVariableOp)' with input shapes: [?,1], [90,64].

Arguments received by SimpleRNNCell.call():
  • sequence=tf.Tensor(shape=(None, 1), dtype=float32)
  • states=('tf.Tensor(shape=(None, 64), dtype=float32)',)
  • training=True
📂 Modelo cnn_AAPL_2025_single no existe. Procediendo a entrenar...
RUNNING MODEL:
Multiple tickers detected: ['AAPL', 'BAM']. Using multi-ticker features.


Architecture: cnn
Reshaping input: n_samples=6211, n_features_total=90
Falló el modelo cnn_AAPL_2025_single: Expected input with 8100 features (horizon=90 * variables=90), but got 90


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_30 (SimpleRNN)       │ (None, 90, 64)         │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_31 (SimpleRNN)       │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_32 (SimpleRNN)       │ (None, 1)              │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,062 (113.53 KB)

...

 Total params: 27,779 (108.52 KB)

 Trainable params: 27,777 (108.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
Modelo cnn_BAM_2025_single ya existe. Saltando...


In [170]:
index_tickers = [
    "^GSPC",          # S&P 500
    "^IXIC",          # Nasdaq
    "^IBEX",          # Ibex 35
    "AMZN",
    "TSLA",
    "GOOG",
    "BAM",
    "BLK",
]

In [171]:

for ticker in index_tickers:
    ticker_single_models(ticker)

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_30 (SimpleRNN)       │ (None, 90, 64)         │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_31 (SimpleRNN)       │ (None, 90, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_32 (SimpleRNN)       │ (None, 1)              │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,062 (113.53 KB)

 Trainable params: 29,060 (113.52 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 27,779 (108.52 KB)

 Trainable params: 27,777 (108.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
Modelo cnn_BLK_2025_single ya existe. Saltando...
